In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


In [8]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv(next(iter(uploaded)))


Saving HSP.csv to HSP.csv


In [9]:
print(df.info())
print(df.describe())
print(df.isnull().sum())
print(f"Duplicated Data: {df.duplicated().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [10]:
cols_to_remove = ['meals_dinner_friend', 'healthy_meal', 'ideal_diet', 'diet_current',
                  'eating_changes', 'food_childhood', 'comfort_food_reasons',
                  'comfort_food_reasons_coded.1', 'eating_changes_coded1',
                  'drink', 'exercise', 'cuisine', 'type_sports']
df.drop(columns=[col for col in cols_to_remove if col in df.columns], inplace=True)


In [11]:
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].apply(lambda x: x.fillna(x.mean()))
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df[num_cols] < (Q1 - 1.5 * IQR)) | (df[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [14]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [15]:
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [16]:
for col in cat_cols:
    if df[col].nunique() <= 10:
        df[col] = LabelEncoder().fit_transform(df[col])
    else:
        df = pd.get_dummies(df, columns=[col], drop_first=True)

In [17]:
threshold = 0.05 * len(df)
for col in cat_cols:
    if col in df.columns:
        counts = df[col].value_counts()
        rare_labels = counts[counts < threshold].index
        df[col] = df[col].replace(rare_labels, "Other")

In [18]:
if "income" in df.columns:
    df["income_log"] = np.log1p(df["income"])
    df["income_sqrt"] = np.sqrt(df["income"])

In [19]:
if "date_of_birth" in df.columns:
    df["date_of_birth"] = pd.to_datetime(df["date_of_birth"])
    df["year"] = df["date_of_birth"].dt.year
    df["month"] = df["date_of_birth"].dt.month
    df["day"] = df["date_of_birth"].dt.day

In [21]:
df.to_csv("HSP_Processed.csv", index=False)